In [86]:
import getpass
import os
from langchain_openai import ChatOpenAI
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [87]:
from datetime import datetime, timedelta
from typing import List, Dict, Optional
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from pydantic import Field
import json
import re
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import dateparser

In [88]:
tasks = {"high":[],"medium":[],"low":[]}
reminders = {"high":[],"medium":[],"low":[]}

In [89]:
from typing_extensions import Annotated
from pydantic import BaseModel
class addTask(BaseModel):
    """Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'."""
    title: Annotated[str,...,"Title of task."]
    deadline: Annotated[str,...,"Deadline of task."] = "today"
    priority:Annotated[str,...,"Priority of task.[High,Medium,Low]"]="medium"

class setReminder(BaseModel):
    """Sets a reminder for a specific task at a given time with priority (high, medium, low)."""
    task_title: Annotated[str,...,"Title of task."]
    reminder_time: Annotated[str,...,"Reminder Time."] = "today"
    priority:Annotated[str,...,"Priority of task.[High,Medium,Low]"]="medium"

class getQuery(BaseModel):
    type:Annotated[str,...,"task or reminder."] 
    priority:Optional[Annotated[str,...,"priority of task[High,Medium,Low]"]]
    time_frame:Optional[Annotated[str,...,"timeframe"]]



In [90]:
# Remove all the old date parsing functions and replace with:

def calculate_deadline(deadline_str: str) -> datetime:
    """Calculate deadline from natural language using dateparser"""
    parsed = dateparser.parse(
        deadline_str,
        settings={
            'RELATIVE_BASE': datetime.now(),
            'PREFER_DATES_FROM': 'future',
            'PREFER_DAY_OF_MONTH': 'first'
        }
    )
    
    if not parsed:
        return datetime.now().replace(hour=23, minute=59, second=59)
    
    # Default to end of day for day-only references
    if not any(c.isdigit() for c in deadline_str if c not in [' ', ',']):
        if deadline_str.lower() in ['today', 'tomorrow']:
            return parsed.replace(hour=23, minute=59, second=59)
        return parsed.replace(hour=9, minute=0, second=0)  # Default 9 AM for day names
    
    return parsed

In [ ]:
@tool(args_schema=addTask)
def add_task(title: str, deadline: str, priority: str) -> Dict:
    """Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'."""
    priority = priority.lower()
    if priority not in tasks:
        return {"error": "Invalid priority"}
    
    try:
        # Simplified date parsing with dateparser
        parsed_date = dateparser.parse(
            deadline,
            settings={
                'RELATIVE_BASE': datetime.now(),
                'PREFER_DATES_FROM': 'future',
                'PREFER_DAY_OF_MONTH': 'first'
            }
        )
        
        # Default to end of day if time not specified
        if not any(c.isdigit() for c in deadline if c not in [' ', ',']):
            parsed_date = parsed_date.replace(hour=23, minute=59, second=59) if deadline.lower() in ['today', 'tomorrow'] \
                         else parsed_date.replace(hour=9, minute=0, second=0)
        
        task = {
            "title": title,
            "deadline": parsed_date.isoformat(),
            "priority": priority,
            "created_at": datetime.now().isoformat()
        }
        tasks[priority].append(task)
        return task
    except Exception as e:
        return {"error": f"Could not parse deadline: {str(e)}"}

@tool(args_schema=setReminder)
def set_reminder(task_title: str, reminder_time: str, priority: str) -> Dict:
    """Sets a reminder for a task. If no time is given (e.g., 'Thursday'), defaults to 9 AM."""
    priority = priority.lower()
    if priority not in reminders:
        return {"error": "Invalid priority"}

    try:
        # Parse natural language time (e.g., "Thursday" → next Thursday 9 AM)
        parsed_time = dateparser.parse(
            reminder_time,
            settings={
                'RELATIVE_BASE': datetime.now(),
                'PREFER_DATES_FROM': 'future',
                'PREFER_DAY_OF_MONTH': 'first',
                'DEFAULT_LANGUAGES': ['en']
            }
        )

        # Default to 9 AM if only a day is specified (e.g., "Thursday")
        if not any(c.isdigit() for c in reminder_time):
            parsed_time = parsed_time.replace(hour=9, minute=0, second=0)

        reminder = {
            "reminder_title": task_title,
            "reminder_time": parsed_time.isoformat(),
            "priority": priority,
            "created_at": datetime.now().isoformat()
        }
        reminders[priority].append(reminder)
        return reminder
    except Exception as e:
        return {"error": f"Could not parse reminder time: {str(e)}"}
@tool(args_schema=getQuery)
def get_query(type: str, priority: str = None, time_frame: str = None) -> Dict:
    """Retrieves tasks or reminders based on query.
    Args:
        type: 'tasks' or 'reminders'
        priority: 'high', 'medium', or 'low' (optional)
        time_frame: Natural language time frame like 'today', 'tomorrow', 'next week' (optional)
    """
    type = type.lower()
    priority = priority.lower() if priority else None
    current_time = datetime.now()

    def filter_by_time(items: List[Dict], time_frame: str) -> List[Dict]:
        if not time_frame:
            return items

        time_frame = time_frame.lower()
        filtered = []
        current_time = datetime.now()
        
        try:
            # Handle "overdue" case
            if time_frame == "overdue":
                for item in items:
                    item_time = datetime.fromisoformat(item.get("deadline") or item.get("reminder_time"))
                    if item_time < current_time:
                        filtered.append(item)
                return filtered

        # Parse the target time frame (e.g., "Thursday" → next Thursday)
            target_date = dateparser.parse(
                time_frame,
                settings={
                    'RELATIVE_BASE': current_time,
                    'PREFER_DATES_FROM': 'future',
                    'DEFAULT_LANGUAGES': ['en']
                }
            )
            
            if not target_date:
                return items

            # Compare dates (ignoring time for day-based queries)
            for item in items:
                item_time = datetime.fromisoformat(item.get("deadline") or item.get("reminder_time"))
                
                if time_frame in ["today", "tomorrow"]:
                    if item_time.date() == target_date.date():
                        filtered.append(item)
                elif time_frame == "week":
                    end_of_week = current_time + timedelta(days=7)
                    if current_time.date() <= item_time.date() <= end_of_week.date():
                        filtered.append(item)
                elif time_frame == "month":
                    end_of_month = current_time + relativedelta(months=1)
                    if current_time.date() <= item_time.date() <= end_of_month.date():
                        filtered.append(item)
                else:  # Day name or specific date (e.g., "Thursday")
                    if item_time.date() == target_date.date():
                        filtered.append(item)
                    
        except Exception:
            return items

        return filtered
        
    # Handle tasks
    if type == "tasks":
        if priority:
            if priority in tasks:
                items = tasks[priority]
                return {"tasks": filter_by_time(items, time_frame)}
            return {"error": "Invalid priority"}
        
        # No priority specified - combine all priorities
        all_tasks = []
        for prio in ["high", "medium", "low"]:
            all_tasks.extend(tasks[prio])
        return {"tasks": filter_by_time(all_tasks, time_frame)}

    # Handle reminders
    elif type == "reminders":
        if priority:
            if priority in reminders:
                items = reminders[priority]
                return {"reminders": filter_by_time(items, time_frame)}
            return {"error": "Invalid priority"}
        
        # No priority specified - combine all priorities
        all_reminders = []
        for prio in ["high", "medium", "low"]:
            all_reminders.extend(reminders[prio])
        return {"reminders": filter_by_time(all_reminders, time_frame)}

    return {"error": "Invalid query type. Must be 'tasks' or 'reminders'"}

In [92]:
from langchain_core.utils.function_calling import convert_to_openai_function
functions = [
    convert_to_openai_function(add_task),
    convert_to_openai_function(set_reminder),
    convert_to_openai_function(get_query)
]

In [93]:
functions

[{'name': 'add_task',
  'description': "Adds a new task with a deadline. Deadline can be in natural language like 'Monday' or 'tomorrow'.",
  'parameters': {'properties': {'title': {'type': 'string'},
    'deadline': {'default': 'today', 'type': 'string'},
    'priority': {'default': 'medium', 'type': 'string'}},
   'required': ['title'],
   'type': 'object'}},
 {'name': 'set_reminder',
  'description': "Sets a reminder for a task. If no time is given (e.g., 'Thursday'), defaults to 9 AM.",
  'parameters': {'properties': {'task_title': {'type': 'string'},
    'reminder_time': {'default': 'today', 'type': 'string'},
    'priority': {'default': 'medium', 'type': 'string'}},
   'required': ['task_title'],
   'type': 'object'}},
 {'name': 'get_query',
  'description': "Retrieves tasks or reminders based on query.\nArgs:\n    type: 'tasks' or 'reminders'\n    priority: 'high', 'medium', or 'low' (optional)\n    time_frame: Natural language time frame like 'today', 'tomorrow', 'next week' (o

In [94]:
model = llm.bind(functions=functions)

In [95]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

In [96]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "add_task": add_task, 
            "set_reminder": set_reminder,
            "get_query":get_query
        }
        return tools[result.tool].run(result.tool_input)

In [97]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [98]:
chain.invoke({"input": "add task to cutting fuits on high priority for thursday."})

{'title': 'cutting fruits',
 'deadline': '2025-04-17T09:00:00',
 'priority': 'high',
 'created_at': '2025-04-16T18:19:54.236836'}

In [99]:
chain.invoke({"input": "add task to cutting fuits on high priority for today."})

{'title': 'cutting fruits',
 'deadline': '2025-04-16T23:59:59.011860',
 'priority': 'high',
 'created_at': '2025-04-16T18:19:56.017270'}

In [100]:
chain.invoke({"input": "Give me reminders for Thursday."})

{'reminders': []}

In [101]:
chain.invoke({"input": "add reminder to cutting fuits on high priority for thursday."})

{'task_title': 'cutting fruits',
 'reminder_time': '2025-04-17T09:00:00',
 'priority': 'high',
 'created_at': '2025-04-16T18:20:00.712923'}

In [102]:
chain.invoke({"input": "Give me reminders for Thursday."})

{'reminders': [{'task_title': 'cutting fruits',
   'reminder_time': '2025-04-17T09:00:00',
   'priority': 'high',
   'created_at': '2025-04-16T18:20:00.712923'}]}

In [103]:
print(tasks)
print(reminders)

{'high': [{'title': 'cutting fruits', 'deadline': '2025-04-17T09:00:00', 'priority': 'high', 'created_at': '2025-04-16T18:19:54.236836'}, {'title': 'cutting fruits', 'deadline': '2025-04-16T23:59:59.011860', 'priority': 'high', 'created_at': '2025-04-16T18:19:56.017270'}], 'medium': [], 'low': []}
{'high': [{'task_title': 'cutting fruits', 'reminder_time': '2025-04-17T09:00:00', 'priority': 'high', 'created_at': '2025-04-16T18:20:00.712923'}], 'medium': [], 'low': []}


In [105]:
chain.invoke({"input": "Give me tasks for tomorrow."})

{'tasks': [{'title': 'cutting fruits',
   'deadline': '2025-04-17T09:00:00',
   'priority': 'high',
   'created_at': '2025-04-16T18:19:54.236836'}]}

In [106]:
chain.invoke({"input": "Give me tasks for Thursday."})

{'tasks': [{'title': 'cutting fruits',
   'deadline': '2025-04-17T09:00:00',
   'priority': 'high',
   'created_at': '2025-04-16T18:19:54.236836'}]}